In [1]:
import requests
from bs4 import BeautifulSoup as soup
import re
import urllib.parse
from urllib.request import Request, urlopen
from urllib.parse import urlparse
import random
import json

In [2]:
def randomAgent():
    
    randomAgents = [
        'Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10.5; en-US; rv:1.9.1b3) Gecko/20090305 Firefox/3.1b3 GTB5',
        'Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10.5; ko; rv:1.9.1b2) Gecko/20081201 Firefox/3.1b2'        ,
        'Mozilla/5.0 (X11; U; SunOS sun4u; en-US; rv:1.9b5) Gecko/2008032620 Firefox/3.0b5',
        'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:2.0b8pre) Gecko/20101114 Firefox/4.0b8pre',
        'Mozilla/5.0 (X11; Linux x86_64; rv:2.0b9pre) Gecko/20110111 Firefox/4.0b9pre'
        'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:2.0b9pre) Gecko/20101228 Firefox/4.0b9pre'
        'Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:2.2a1pre) Gecko/20110324 Firefox/4.2a1pre'
        'Mozilla/5.0 (X11; U; Linux amd64; rv:5.0) Gecko/20100101 Firefox/5.0 (Debian)'
        'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:6.0a2) Gecko/20110613 Firefox/6.0a2'
        'Mozilla/5.0 (X11; Linux i686 on x86_64; rv:12.0) Gecko/20100101 Firefox/12.0'
                   ]
    
    return random.choice(randomAgents)

In [3]:
def scrape(url):
        
        req = Request(url, headers={'User-Agent' : randomAgent()})
        
        webpage = urlopen(req).read()
        return webpage

In [4]:
wiki_skeletal_muscles = 'https://en.wikipedia.org/wiki/List_of_skeletal_muscles_of_the_human_body'

In [5]:
page = scrape(wiki_skeletal_muscles)

In [6]:
def scrapeByTag(webpage, tag):
    
        page_soup = soup(webpage,'html.parser')
        taglist = page_soup.findAll(tag)
        
        return taglist

In [7]:
def scrapeByClassName(webpage, tag, classname):

        page_soup = soup(webpage,'html.parser')
        classlist = page_soup.findAll(tag, class_=classname)
        
        return classlist

In [8]:
def formatText(text, i): 
    
        text = str(text)
    
        cleaned = re.sub(r"<{}>|<{}>", "", text).format(i)
        
        return cleaned

In [9]:
def toJSON(title, obj, location):
    with open(location + '/' + title + '.json', 'w', encoding='utf-8') as f:
        json.dump(obj, f, ensure_ascii=False, indent=4)
    return

In [10]:
tables = scrapeByClassName(page, 'table', 'wikitable')

In [11]:
def getHeaders(head):
    
    headings = []
    for item in head.find_all("th"):
        item = (item.text).rstrip("\n")
        headings.append(item)
        
    return headings

In [12]:
def getRows(rows):
    
    all_rows = [] 
    
    for row_num in range(len(rows)):
        row = []
        for row_item in rows[row_num].find_all("td"):
            aa = re.sub("(\xa0)|(\n)|,","",row_item.text)
            row.append(aa)
        all_rows.append(row)
        
    return all_rows

In [13]:
def createObjects(headers, rows):
    
    objects = []
    
    for row in rows:
        
        if len(row) == len(headers):
            
            obj = {}
        
            for i in range(len(headers)):

                title = headers[i]
                data = row[i]

                obj[title] = data
            
            objects.append(obj)
            
        else:
            continue
            
    return objects

In [14]:
def extractMusclesData(page):
    
    muscles = []
    
    tables = scrapeByClassName(page, 'table', 'wikitable')
    
    for table in tables:
        
        body = table.find_all("tr")
        muscles += createObjects(getHeaders(body[0]), getRows(body[1:]))
        
    return muscles

In [17]:
toJSON('skeletalMuscles', extractMusclesData(page), 'Muscles')